In [1]:
# System wspomagania decyzji inwestorskich
# Paweł Kowalski

In [2]:
# sprawdzenieczasu działania programu
import time as tm
start_time = tm.time()

In [3]:
# import bibliotek
import yfinance as yf
from datetime import date
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib 
import sklearn  
import statsmodels 
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.dates as mdates
from sklearn.linear_model import LinearRegression
from plik_tickers import tickers 

In [4]:
# Sprawdzanie wersji Phytona
import sys 
print(f"Python: {sys.version}")

Python: 3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]


In [5]:
# Sprawdzanie wersji bibliotek
print(f"yfinance: {yf.__version__}")
print(f"numpy: {np.__version__}")
print(f"matplotlib: {matplotlib.__version__}")
print(f"tensorflow: {tf.__version__}")
print(f"scikit-learn: {sklearn.__version__}")
print(f"seaborn: {sns.__version__}")
print(f"statsmodels: {statsmodels.__version__}")

yfinance: 0.1.67
numpy: 1.24.3
matplotlib: 3.7.1
tensorflow: 2.11.0
scikit-learn: 1.2.2
seaborn: 0.12.2
statsmodels: 0.13.5


In [6]:
# ustawienie ziarna i czyszczenie
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [7]:
# wybór tickers'a , tu musi być np. lista rozwijana
for symbol, name in tickers.items():
    print(f"Symbol: {symbol} - Name: {name}")

Symbol: ES=F - Name: S&P 500
Symbol: YM=F - Name: Mini Dow Jones Indus
Symbol: NQ=F - Name: Nasdaq 100
Symbol: RTY=F - Name: E-mini Russell 2000 Index Futur
Symbol: ZB=F - Name: U.S. Treasury Bond Futures
Symbol: ZN=F - Name: 10-Year T-Note Futures
Symbol: ZF=F - Name: Five-Year US Treasury Note Futu
Symbol: ZT=F - Name: 2-Year T-Note Futures
Symbol: GC=F - Name: Gold
Symbol: MGC=F - Name: Micro Gold Futures
Symbol: SI=F - Name: Silver
Symbol: PL=F - Name: Platinum
Symbol: HG=F - Name: Copper
Symbol: PA=F - Name: Palladium
Symbol: CL=F - Name: Crude Oil
Symbol: HO=F - Name: Heating Oil
Symbol: NG=F - Name: Natural Gas
Symbol: RB=F - Name: RBOB Gasoline
Symbol: BZ=F - Name: Brent Crude Oil
Symbol: B0=F - Name: Mont Belvieu LDH Propane
Symbol: ZC=F - Name: Corn Futures, Dec-2024
Symbol: ZO=F - Name: Oat Futures, Dec-2024
Symbol: KE=F - Name: KC HRW Wheat Futures
Symbol: ZR=F - Name: Rough Rice Futures, Sep-2024
Symbol: ZM=F - Name: S&P Composite 1500 ESG Tilted I
Symbol: ZL=F - Name: Soy

In [8]:
# pobranie danych ze strony Yahoo (po testach wpisać np. rok  2000 !
start = "2004-01-01"
end = date.today().strftime("%Y-%m-%d")
ticker = "BZ=F" # możliwość wyboru np 10
data = yf.download(ticker, start=start, end=end)
print(data.shape)

[*********************100%***********************]  1 of 1 completed
(4197, 6)


In [9]:
# Kontrola jakości danych
print(data.info())
print(data.describe())
print('head', data.head(5))
print('tail', data.tail(5))
print(type(data))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4197 entries, 2007-07-30 to 2024-06-25
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       4197 non-null   float64
 1   High       4197 non-null   float64
 2   Low        4197 non-null   float64
 3   Close      4197 non-null   float64
 4   Adj Close  4197 non-null   float64
 5   Volume     4197 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 229.5 KB
None
              Open         High          Low        Close    Adj Close  \
count  4197.000000  4197.000000  4197.000000  4197.000000  4197.000000   
mean     78.716612    79.680217    77.699061    78.720000    78.720000   
std      24.676631    24.794818    24.546994    24.694999    24.694999   
min      19.559999    21.270000    16.000000    19.330000    19.330000   
25%      59.200001    60.240002    58.349998    59.270000    59.270000   
50%      76.879997    77.790001    75.830002    77.0500